# Daily Glucose Overlay

This notebook generates daily glucose overlay charts from blood glucose readings held in the Health Tracker database and retrieved via the Health Tracker Web Service:

- A daily overlay or "spaghetti" chart with one line per day
- A "percentile" chart shading between upper and lower percentiles

Both charts include the median line.

Before attempting to run the notebook:

- Make sure the variables defined in "config.ipynb" are set correctly
- Set the reporting date range and export options in the first code cell

In [ ]:
from datetime import date, timedelta, datetime

# Retrieve all-time blood glucose readings
start = date(2025, 8, 4)
end = start + timedelta(days=11)

# Exclusion date ranges - this should be a list of tuples in which each tuple has two
# members, a start date and time and end date and time in that order
exclusions = [
    (datetime(2025, 8, 6, 6, 52, 0), datetime(2025, 8, 6, 23, 59, 59)),
    (datetime(2025, 8, 12, 15, 7, 0), datetime(2025, 8, 12, 16, 52, 59))
]

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
# Set the percentile limits
lower_percentile = 0
upper_percentile = 100

# Set the Y-axis limits
y_min = 3
y_max = 15

In [ ]:
%run ../api.ipynb
%run ../config.ipynb
%run ../export.ipynb

In [ ]:
# Log in to the service, get the person ID and retrieve the data
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_blood_glucose_measurements(url, token, person_id, start, end)

# Remove any excluded ranges
df = remove_records_for_date_ranges(df, exclusions)

# Preview the data
df.head()

In [ ]:
# Rename the existing date column, that contains a date and time and round the timestamp to the
# nearest 5 minutes to align the data (failing to do this means the pivot won"t work well when
# plotted)
df.rename(columns={ "date": "timestamp" }, inplace=True)
df["timestamp"] = df["timestamp"].dt.round("5min")

# Create columns for date and time-of-day
df["date"] = df["timestamp"].dt.date
df["time"] = df["timestamp"].dt.strftime("%H:%M")

# Pivot the data where rows are the time of day and columns are the date
pivot_df = df.pivot(index="time", columns="date", values="level")

# Preview the data
pivot_df.head()

In [ ]:
# Calculate the median line
median_glucose = pivot_df.median(axis=1)

# Calculate percentile bands for the shaded plot
q10 = pivot_df.quantile(lower_percentile / 100, axis=1)
q90 = pivot_df.quantile(upper_percentile / 100, axis=1)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Plot a spaghetti plot in which each line is 1 day
ax = pivot_df.plot(figsize=(12, 8), legend=False, alpha=0.7, linewidth=1)

# Add the median line
plt.plot(median_glucose.index, median_glucose.values, color="black", label="Median", linewidth=2)

# Add title and labels
ax.set_title("Daily Glucose Overlay")
ax.set_xlabel("Time of Day")
ax.set_ylabel("Glucose (mmol/L)")
plt.ylim(y_min, y_max)
plt.xticks(rotation=45)
plt.grid(True)

# Create a legend below the plot
plt.legend(
    title="Date",
    bbox_to_anchor=(0.5, -0.2),
    loc="upper center",
    ncol=6,
    fontsize="small",
    title_fontsize="small"
)

plt.tight_layout()

# Export to PNG or PDF, if required
export_chart("glucose_overlay", "Daily", chart_export_format)

plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Set x-ticks to every hour (12 x 5 minutes = 60 minutes = 1 hour)
xticks_to_show = pivot_df.index[::12]

# Shade between the percentiles
plt.figure(figsize=(12, 8))
percentile_label = f"{lower_percentile} - {upper_percentile} Percentile"
plt.fill_between(q10.index, q10, q90, color="lightblue", alpha=0.7, label=percentile_label)

# Add the median line
plt.plot(median_glucose.index, median_glucose.values, color="black", label="Median", linewidth=1)

# Add title and labels
plt.title("Daily Glucose Overlay")
plt.xlabel("Time of Day")
plt.ylabel("Glucose (mmol/L)")
plt.ylim(y_min, y_max)
plt.xticks(ticks=xticks_to_show, rotation=90)
plt.grid(True)

# Create a legend below the plot
plt.legend(
    title="Date",
    bbox_to_anchor=(0.5, -0.2),
    loc="upper center",
    ncol=6,
    fontsize="small",
    title_fontsize="small"
)

plt.tight_layout()

# Export to PNG or PDF, if required
export_chart("glucose_overlay", "Percentile", chart_export_format)

plt.show()
